New in version 1B: can now specify `in_season`, which is None (meaning use whole year), or 230 (thru DOY 230 which is mid-August), or 160 (thru DOY 160 which is early June)

In [15]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

In [12]:
71 // 5

14

need:
depending on scheme name, give me a different number of features.

need:
in_season=

determine *nbr_features* based on combo of scheme_name and in_season.

get *nbr_refl_feats* or *nrf*

pdsize = int(scheme_name[:-3])
nrf = ((in_season - 90) // pdsize) * 6 + 6

Note:
- for 14day and 160, nrf =
- for 14day and 230, nrf =
- for 5day and 160, nrf =
- for 5day and 230, nrf =

In [37]:
def create_X_y_single_year(tile,year,scheme_name,crop_of_interest_id,
                          in_season=None):
    coiid = crop_of_interest_id
    
    refl = np.load(f'../data/composited_interpolated/Refl_{tile}_{year}_{scheme_name}.npy')

    # Determine nrf (number of reflectance features)
    if in_season not in [160, 230, None]:
        print('Please change in_season to 160, 230, or None.')
        return None
    if in_season is None:
        nrf = refl.shape[1]
    if in_season in [160, 230]:
        pdsize = int(scheme_name[:-3])
        nrf = ((in_season - 90) // pdsize) * 6 + 6
        
    crop = []
    for y in range(year-4,year+1):
        crop.append(np.load(f'../data/processed_crop/Crop_{tile}_{y}.npy')==coiid)

    X = np.column_stack([refl[:,:nrf]] + crop[:-1])
    y = crop[-1]
    
    return X, y

In [38]:
def create_X_y_multiyear(tile,
                      years,
                      scheme_name,
                      crop_of_interest_id,
                        in_season=None):
    X_list = []
    y_list = []
    
    for y in years:
        X, y = create_X_y_single_year(tile,y,scheme_name,
                                      crop_of_interest_id,
                                     in_season)
        X_list.append(X)
        y_list.append(y)
    
    X = np.concatenate(X_list)
    y = np.concatenate(y_list)
    
    return X, y

In [39]:
def create_X_y(tile,
              years,
              scheme_name,
              crop_of_interest_id,
              in_season=None
              ):
    if type(years)==int:
        return create_X_y_single_year(tile,
                                      years,
                                      scheme_name,
                                      crop_of_interest_id,
                                      in_season)
    
    return create_X_y_multiyear(tile,
                              years,
                              scheme_name,
                              crop_of_interest_id,
                              in_season)

In [42]:
def fit_predict_report(model_name,
                      model,
                      training_sample_size,
                      tile,
                      years,
                      scheme_name,
                      crop_of_interest_id,
                      in_season
                      ):

    conf = []

    for val_year in years:
        print('Starting a fold...')
        print('> Assembling the datasets')
        train_years = [yr for yr in range(2018,2023) if yr!=val_year]

        X_train0, y_train0 = create_X_y(tile,train_years,
                                      scheme_name,crop_of_interest_id,
                                       in_season)
        X_val, y_val = create_X_y(tile,val_year,
                                  scheme_name,crop_of_interest_id,
                                  in_season)

        if training_sample_size is not None:
            X_train, X_surplus, y_train, y_surplus = train_test_split(X_train0,
                                                                     y_train0,
                                                                     train_size=training_sample_size,
                                                                     random_state=19)
        if training_sample_size is None:
            X_train, y_train = X_train0, y_train0

        print('> Fitting the model on the training set')
        model.fit(X_train, y_train)
        print('> Predicting on the validation set')
        pred = model.predict(X_val)

        print('> Recording performance metrics')
        act = y_val
        ActPred_00 = sum((act==0) & (pred==0))
        ActPred_01 = sum((act==0) & (pred==1))
        ActPred_10 = sum((act==1) & (pred==0))
        ActPred_11 = sum((act==1) & (pred==1))
        conf_1yr = [ActPred_00, ActPred_01, ActPred_10, ActPred_11]

        conf.append(conf_1yr)
        print('Finished a fold.')

    carr = np.array(conf)
    totals = carr.sum(axis=0)

    carr = np.row_stack([carr,totals])

    # above we added the totals row
    # now we need to add the columns for precision and recall

    # create dataframe
    cdf = pd.DataFrame(data = carr,
                      index = [f'ValYear{yr}' for yr in years]+['Total'],
                      columns = ['ActPred_00', 'ActPred_01', 
                                 'ActPred_10', 'ActPred_11']
                      )

    cdf['Precision'] = cdf.ActPred_11 / (cdf.ActPred_01 + cdf.ActPred_11)
    cdf['Recall'] = cdf.ActPred_11 / (cdf.ActPred_10 + cdf.ActPred_11)
    
    param_strings = [f'# {k}: {p[k]}' for k in p.keys() if k!='years']
    comment = '\n'.join(param_strings) + '\n' 
    with open(f'../data/results/{model_name}.csv', 'a') as f:
        f.write(comment)
        cdf.to_csv(f)
    
    print(f'Find results in ../data/results/{model_name}.csv')

## BROWN PAPER BAG v2.0

draft below.

In [44]:
## Parameters dictionary p
p = {

## SPECIFY MODEL ##
'model_name': 'BrownBag06--SGDLogistic',
'model': make_pipeline(StandardScaler(),
                       SGDClassifier(loss='log_loss')),
'training_sample_size': 0.01,

## SPECIFY TILE AND SCHEME ##
'tile': '10SFH',
'years': [2018, 2019, 2020, 2021, 2022],
'scheme_name': '14day',
'crop_of_interest_id': 75, # Almonds 
'in_season': None
}

In [45]:
fit_predict_report(**p) # run with the above parameters

Starting a fold...
> Assembling the datasets


KeyboardInterrupt: 

I think the above will work!



In [54]:
import os

In [59]:
# for next notebook, when I do this in a for loop:

f'../data/results/{csv_name}' # before ANYTHING else,
# , check that this model name has no csv yet

'../data/results/BrownBag06--SGDLogistic_0.01_10SFH_14day_75_None.csv'

In [56]:
# produce file_name:
exempt = ['years', 'model']
param_value_strings = [f'{p[k]}' for k in p.keys() if k not in exempt]
csv_name = '_'.join(param_value_strings) +'.csv'

In [57]:
csv_name

'BrownBag06--SGDLogistic_0.01_10SFH_14day_75_None.csv'

In [58]:
# check whether care_about exists
# before ANYTHING else,
# , check that this file_name has no csv yet

if csv_name in os.listdir('../data/results/'):
    continue # continue to the next iteration


SyntaxError: 'continue' not properly in loop (974275021.py, line 6)

What the saving will look like (end of for-loop):

In [ ]:
    param_strings = [f'# {k}: {p[k]}' for k in p.keys() if k!='years']
    comment = '\n'.join(param_strings) + '\n' 
    with open(f'../data/results/{csv_name}', 'a') as f:
        f.write(comment)
        cdf.to_csv(f)
    
    print(f'Find results in ../data/results/{csv_name}')